# Shape Classifier
A simple CNN model to classify geometric shapes (circle, rectangle, square, triangle) using TensorFlow Lite.

In [ ]:
# Imports
import tensorflow as tf
import numpy as np
from PIL import Image
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# Variables
DATASET_PATH = "dataset/train"
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 20

In [ ]:
# Training
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=42
)

In [ ]:
# Validation
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=42
)

In [ ]:
class_names = train_ds.class_names

train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
val_ds = val_ds.map(lambda x, y: (x / 255.0, y))

In [ ]:
# Generating Model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(class_names))
])

In [ ]:
# Compiling and Training
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

In [ ]:
# Conversion
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('models/model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
# Loading model
interpreter = tf.lite.Interpreter(model_path="models/model.tflite")
interpreter.allocate_tensors()

# Load image
test_image = "image.jpg"
img = Image.open(test_image).resize((IMG_SIZE, IMG_SIZE))
img = np.expand_dims(np.array(img) / 255.0, axis=0).astype(np.float32)

In [ ]:
interpreter.set_tensor(interpreter.get_input_details()[0]['index'], img)
interpreter.invoke()
prediction = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

predicted_class = np.argmax(prediction)
predicted_class_name = class_names[predicted_class].title()
confidence = np.max(tf.nn.softmax(prediction)) * 100

print(f"Prediction: {predicted_class_name} ({confidence:.1f}%)")